In [ ]:
## This notebook selects the youtube wav samples

In [ ]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile
import sys

In [ ]:
# Project directory
project_name = 'amsterdam_custom_samples'

In [ ]:
# Set your serval data folder (should be correctly set already)
serval_data_folder = "../data"
project_data_folder = serval_data_folder + '/' + project_name

In [ ]:
# Set output csv files
input_balanced_wav_samples_enumerated_filepath   = serval_data_folder + "/csv_files/wav_samples_youtube_balanced_all_enumerated_and_labeled.csv"
input_unbalanced_wav_samples_enumerated_filepath = serval_data_folder + "/csv_files/wav_samples_youtube_unbalanced_all_enumerated_and_labeled.csv"
input_eval_wav_samples_enumerated_filepath       = serval_data_folder + "/csv_files/wav_samples_youtube_eval_all_enumerated_and_labeled.csv"

# Custom
input_custom_wav_samples_enumerated_filepath      = serval_data_folder + "/wav_samples_custom/wav_samples_custom_all_enumerated_and_labeled.csv"

In [ ]:
# Project input and output files
input_selected_classes_filepath = project_data_folder + '/csv_files/input_selected_wav_samples.csv'
target_selected_classes_filepath = project_data_folder + '/csv_files/output_selected_wav_samples.csv'

In [ ]:
# Load selected classes from csv
df_selected_classes = pd.read_csv(input_selected_classes_filepath, sep=",")

# Print
df_selected_classes

In [ ]:
# Load youtube wav samples and classes
df_balanced_wav_samples_enumerated   = pd.read_csv(input_balanced_wav_samples_enumerated_filepath, sep=";")
df_unbalanced_wav_samples_enumerated = pd.read_csv(input_unbalanced_wav_samples_enumerated_filepath, sep=";")
df_eval_wav_samples_enumerated       = pd.read_csv(input_eval_wav_samples_enumerated_filepath, sep=";")

# Remove unnamed index column if it exists
df_balanced_wav_samples_enumerated   = df_balanced_wav_samples_enumerated.loc  [:, ~df_balanced_wav_samples_enumerated.columns.str.match('Unnamed')]
df_unbalanced_wav_samples_enumerated = df_unbalanced_wav_samples_enumerated.loc[:, ~df_unbalanced_wav_samples_enumerated.columns.str.match('Unnamed')]
df_eval_wav_samples_enumerated       = df_eval_wav_samples_enumerated.loc      [:, ~df_eval_wav_samples_enumerated.columns.str.match('Unnamed')]

# Combine
df_youtube_samples = df_balanced_wav_samples_enumerated.append(df_unbalanced_wav_samples_enumerated).append(df_eval_wav_samples_enumerated)

# Print
df_youtube_samples.head()

In [ ]:
# Load custom wav samples and classes
df_custom_wav_samples_enumerated = pd.read_csv(input_custom_wav_samples_enumerated_filepath, sep=";")

# Remove unnamed index column if it exists
df_custom_wav_samples_enumerated = df_custom_wav_samples_enumerated.loc  [:, ~df_custom_wav_samples_enumerated.columns.str.match('Unnamed')]

# Print
df_custom_wav_samples_enumerated.head()

In [ ]:
# Combine
df_wav_samples_enumerated = df_youtube_samples.append(df_custom_wav_samples_enumerated)

In [ ]:
# Filter all csv with interesting classes
df_selected_samples = df_wav_samples_enumerated.loc[df_wav_samples_enumerated.mid.isin(df_selected_classes.mid)]

In [ ]:
display(df_selected_samples.groupby(['label', 'mid', 'display_name']).agg(['nunique']))

In [ ]:
# Write csv out again to 
df_selected_samples.to_csv(target_selected_classes_filepath, sep=';')